In [35]:
import requests

song fetching code from Nick Pai: https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2

In [36]:
GENIUS_API_TOKEN='nt4zNQ39MV8EGc7_AM6Fa2NMnAJB6LKvz3KOXnK5jfp-ZdisWRmfNja9BGRLwBoV'

In [113]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
songs = request_song_url('Taylor Swift', 500)
songs

Found 500 songs by Taylor Swift


['https://genius.com/Taylor-swift-all-too-well-10-minute-version-taylors-version-from-the-vault-lyrics',
 'https://genius.com/Taylor-swift-cardigan-lyrics',
 'https://genius.com/Taylor-swift-all-too-well-10-minute-version-taylors-version-live-acoustic-lyrics',
 'https://genius.com/Taylor-swift-exile-lyrics',
 'https://genius.com/Taylor-swift-lover-lyrics',
 'https://genius.com/Taylor-swift-the-1-lyrics',
 'https://genius.com/Taylor-swift-august-lyrics',
 'https://genius.com/Taylor-swift-look-what-you-made-me-do-lyrics',
 'https://genius.com/Taylor-swift-betty-lyrics',
 'https://genius.com/Taylor-swift-champagne-problems-lyrics',
 'https://genius.com/Taylor-swift-end-game-lyrics',
 'https://genius.com/Taylor-swift-blank-space-lyrics',
 'https://genius.com/Taylor-swift-me-lyrics',
 'https://genius.com/Taylor-swift-you-need-to-calm-down-lyrics',
 'https://genius.com/Taylor-swift-my-tears-ricochet-lyrics',
 'https://genius.com/Taylor-swift-ready-for-it-lyrics',
 'https://genius.com/Taylor-

In [45]:
from bs4 import BeautifulSoup

In [162]:
html = BeautifulSoup(requests.get('https://genius.com/Taylor-swift-august-lyrics').text)
lyrics = html.find_all("div", {'class': 'Lyrics__Container-sc-1ynbvzw-6 YYrds'})
full = ""
for l in lyrics:
    x = l.get_text(separator="\n").split("[")
    for sect in x:
        full += sect + "\n" + "["
    full = full.strip("[")

In [153]:
full

'\n[Verse 1]\nSalt air, and the rust on your door\nI never needed anything more\nWhispers of "Are you\u2005sure?"\n"Never\u2005have I ever\u2005before"\n\n[Chorus]\nBut I can see us lost\u2005in the memory\nAugust slipped away into a moment in time\n\'Cause it was never mine\nAnd I can see us twisted in bedsheets\nAugust sipped away like a bottle of wine\n\'Cause you were never mine\n\n[Verse 2]\nYour back beneath the sun\nWishin\' I could write my name on it\nWill you call when you\'re back at school?\nI remember thinkin\' I had you\n\n[Chorus]\nBut I can see us lost in the memory\nAugust slipped away into a moment in time\n\'Cause it was never mine\nAnd I can see us twisted in bedsheets\nAugust sipped away like a bottle of wine\n\'Cause you were never mine\n\n[Bridge]\nBack when we were still changin\' for the better\nWanting was enough\nFor me, it was enough\nTo live for the hope of it all\nCancel plans just in case you\'d call\nAnd say, "Meet me behind the mall"\nSo much for summer

In [156]:
def get_song(url):
    html = BeautifulSoup(requests.get(url).text)
    lyrics = html.find_all("div", {'class': 'Lyrics__Container-sc-1ynbvzw-6 YYrds'})
    full = ""
    for sect in lyrics:
        parts = sect.get_text(separator="\n").split("[")
        for pt in parts:
            full += pt + "\n" + "["
        full = full.strip("[")
    return full

In [157]:
omit = ["acoustic", "live-2011", "demo", "annotated", "long-pond", "live-from", "remix"]
used = 0
all_text = ""
while len(all_text) < 1000000 and used < len(songs):
    song = songs[used]
    use_song = True
    for keyword in omit:
        if keyword in song:
            use_song = False
            break
    
    if use_song:
        try:
            all_text += get_song(song)
            all_text += "\n"

        except:
            print("Could not get lyrics element from " + song)
    else:
        print("Chose to omit " + song)
    used += 1

Chose to omit https://genius.com/Taylor-swift-all-too-well-10-minute-version-taylors-version-live-acoustic-lyrics
Chose to omit https://genius.com/Taylor-swift-bad-blood-remix-lyrics
Chose to omit https://genius.com/Taylor-swift-lover-remix-lyrics
Chose to omit https://genius.com/Taylor-swift-reputation-prologue-annotated
Chose to omit https://genius.com/Taylor-swift-red-original-demo-recording-lyrics
Chose to omit https://genius.com/Taylor-swift-i-knew-you-were-trouble-intro-annotated
Chose to omit https://genius.com/Taylor-swift-back-to-december-apologize-youre-not-sorry-live-2011-lyrics
Chose to omit https://genius.com/Taylor-swift-unreleased-songs-discography-list-annotated
Chose to omit https://genius.com/Taylor-swift-if-youre-anything-like-me-poem-annotated
Chose to omit https://genius.com/Taylor-swift-why-she-disappeared-poem-annotated
Chose to omit https://genius.com/Taylor-swift-speak-now-liner-notes-annotated
Chose to omit https://genius.com/Taylor-swift-red-liner-notes-annot

In [158]:
len(all_text)

706290

In [159]:
f = open('tswiftlyrics.txt', 'w')
f.write(all_text)

706290